In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv(r'C:\Users\Bharathwaj\Sports\IPL_ball_by_ball_updated.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225953 entries, 0 to 225952
Data columns (total 22 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   match_id                225953 non-null  int64  
 1   season                  225953 non-null  int64  
 2   start_date              225953 non-null  object 
 3   venue                   225953 non-null  object 
 4   innings                 225953 non-null  int64  
 5   ball                    225953 non-null  float64
 6   batting_team            225953 non-null  object 
 7   bowling_team            225953 non-null  object 
 8   striker                 225953 non-null  object 
 9   non_striker             225953 non-null  object 
 10  bowler                  225953 non-null  object 
 11  runs_off_bat            225953 non-null  int64  
 12  extras                  225953 non-null  int64  
 13  wides                   7026 non-null    float64
 14  noballs             

In [3]:
kohli = df[(df['striker'] == "RG Sharma")]

In [5]:
#df_mali.info()

In [4]:
kohli = kohli.query('wicket_type != "run out"')

In [5]:
batsman_stats = kohli.groupby('bowler').agg({
    'ball': 'count',               # Total number of balls faced
    'runs_off_bat': 'sum',         # Total number of runs scored
    'player_dismissed': 'count'    # Number of times dismissed
}).reset_index()

# Renaming columns for clarity
batsman_stats.columns = ['bowler', 'balls_faced', 'runs_scored', 'times_dismissed']

In [6]:
batsman_stats['strike_rate'] = (batsman_stats['runs_scored'] / batsman_stats['balls_faced']) * 100

In [7]:
batsman_stats.to_csv('rohit_b2b.csv' , sep=',', index = True, encoding='utf-8')

In [10]:
# Define the Pandas UDF
def get_player_stats(df, batsman, bowler):
    filtered_df = df[(df['striker'] == batsman) & (df['bowler'] == bowler)]
    runs_scored = filtered_df['runs_off_bat'].sum()
    balls_bowled = filtered_df.shape[0]
    wickets_taken = filtered_df.loc[df['wicket_type'].notnull(), 'wicket_type'].count()
    strike_rate = (runs_scored / balls_bowled) * 100
    dot_balls =  filtered_df.loc[(df['runs_off_bat'] == 0), 'runs_off_bat'].count()
    fours = filtered_df.loc[(df['runs_off_bat'] == 4), 'runs_off_bat'].count()
    sixes = filtered_df.loc[(df['runs_off_bat'] == 6), 'runs_off_bat'].count()
    average = (runs_scored/wickets_taken)
    dot_percentage = (dot_balls/balls_bowled) * 100
    boundary_percentage = ((fours + sixes)/balls_bowled) * 100
    
    stats = {
        "runs": runs_scored,
        "balls": balls_bowled,
        "wickets": wickets_taken,
        "dots": dot_balls,
        "strike_rate": strike_rate,
        "4s": fours,
        "6s": sixes,
        "avg": average,
        "dot%": dot_percentage,
        "boundary%": boundary_percentage}
    return stats

In [11]:
batsman = "KA Pollard"
bowler = "Rashid Khan"
head2head = get_player_stats(df, batsman, bowler)

head2head

{'runs': 28,
 'balls': 48,
 'wickets': 2,
 'dots': 29,
 'strike_rate': 58.333333333333336,
 '4s': 1,
 '6s': 1,
 'avg': 14.0,
 'dot%': 60.416666666666664,
 'boundary%': 4.166666666666666}

In [16]:
batsman = "KA Pollard"
bowler = "SM Curran"
head2head = get_player_stats(df, batsman, bowler)

head2head

{'runs': 54,
 'balls': 24,
 'wickets': 1,
 'dots': 9,
 'strike_rate': 225.0,
 '4s': 3,
 '6s': 6,
 'avg': 54.0,
 'dot%': 37.5,
 'boundary%': 37.5}